In [49]:
import pandas as pd
import numpy as np
%matplotlib inline

In [50]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Allerlei Data Exploration (visualisatie) to get a feel about the data

# Data Cleaning

In [51]:
train.dropna(subset = ['Gender', 'Married', 'Dependents'], inplace=True)
train['Self_Employed'].fillna('No', inplace=True)
train['LoanAmount'].fillna(train['LoanAmount'].mean(), inplace=True)
train['Loan_Amount_Term'].fillna(train['Loan_Amount_Term'].mean(), inplace=True)
train.dropna(subset =['Credit_History'], inplace=True)
train['Credit_History'].dropna(inplace=True)

In [52]:
train.isnull().sum()

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

In [53]:
test.dropna(inplace = True)

In [54]:
test.isnull().sum()

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
dtype: int64

In [55]:
#Transforming all Categorical Variables to Numeric variables
# LabelEncoder: Encode labels with value between 0 and n_classes-1
from sklearn.preprocessing import LabelEncoder
var_mod = ['Gender','Married','Dependents','Education','Self_Employed','Property_Area','Loan_Status']
le = LabelEncoder()
for i in var_mod:
    train[i] = le.fit_transform(train[i])

In [56]:
from sklearn.preprocessing import LabelEncoder
var_mod = ['Gender','Married','Dependents','Education','Self_Employed','Property_Area']
for i in var_mod:
    test[i] = le.fit_transform(test[i])

# Defining the model function

In [62]:
#Import models from scikit learn module:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold   #For K-fold cross validation
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn import metrics

#Generic function for making a classification model and accessing performance:
def classification_model(model, data, predictors, outcome):
  #Fit the model:
  model.fit(data[predictors],data[outcome])
  
  #Make predictions on training set:
  predictions = model.predict(data[predictors])
  
  #Print accuracy
  accuracy = metrics.accuracy_score(predictions,data[outcome])
  print ("Accuracy : %s" % "{0:.3%}".format(accuracy))

  #Perform k-fold cross-validation with 5 folds
  kf = KFold(data.shape[0], n_folds=5)
  error = []
  for train, test in kf:
    # Filter training data
    train_predictors = (data[predictors].iloc[train,:])
    
    # The target we're using to train the algorithm.
    train_target = data[outcome].iloc[train]
    
    # Training the algorithm using the predictors and target.
    model.fit(train_predictors, train_target)
    
    #Record error from each cross-validation run
    error.append(model.score(data[predictors].iloc[test,:], data[outcome].iloc[test]))
 
  print ("Cross-Validation Score : %s" % "{0:.3%}".format(np.mean(error)))

  #Fit the model again so that it can be refered outside the function:
  model.fit(data[predictors],data[outcome]) 

In [58]:
outcome_var = 'Loan_Status'
model = LogisticRegression()
predictor_var = ['Credit_History']
classification_model(model, train, predictor_var,outcome_var)

Accuracy : 81.192%
Cross-Validation Score : 81.198%


In [59]:
#Model is nu gebasseerd op credit history only, als je hier meer IV wilt toevoegen, dan moet je boven bij de predictor_var 
#dat ook inbouwen, anders krijg je error.

prediction = model.predict(test[['Credit_History']])

In [60]:
test['Loan_Status'] = prediction

In [61]:
test['Loan_Status']

0      1
1      1
2      1
4      1
5      1
6      1
7      0
9      1
10     1
14     1
15     1
16     1
17     1
18     1
19     1
20     1
21     1
23     1
24     1
25     0
27     1
29     1
30     1
31     1
32     1
33     1
34     1
35     0
37     1
38     1
      ..
332    1
333    1
334    1
335    1
337    1
338    1
339    0
340    1
341    1
342    1
343    1
344    1
345    1
346    0
347    1
348    1
349    1
350    1
352    1
353    1
354    0
355    1
356    1
357    1
359    1
361    1
362    1
363    1
365    1
366    1
Name: Loan_Status, Length: 289, dtype: int64